# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 100
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
test_loop = 1

## Load data

In [5]:
data_name = 'adult-race'

In [6]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [7]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [8]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 112), (64, 1), (64, 5)), types: (tf.float64, tf.float64, tf.float64)>

In [9]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [10]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [11]:
header = "model_name", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [12]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.14257341623306274 | 0.534588634967804 | 0.5343420505523682 | 0.9201741218566895 | 0.6934681697612732 | 0.6414261823742397 | -101.54302055702918
> 2 | 0.14125140011310577 | 0.5168285369873047 | 0.4653715193271637 | 0.9201741218566895 | 0.8258454907161804 | 0.7178519111848004 | -101.54302055702918
> 3 | 0.14021243155002594 | 0.5135079622268677 | 0.46435338258743286 | 0.9201741218566895 | 0.8343418435013262 | 0.7178519111848004 | -101.54302055702918
> 4 | 0.13851915299892426 | 0.508553147315979 | 0.46497833728790283 | 0.9201741218566895 | 0.8362897877984085 | 0.7178519111848004 | -101.54302055702918
> 5 | 0.1372690349817276 | 0.5049388408660889 | 0.46565860509872437 | 0.9201741218566895 | 0.8384864058355438 | 0.7178519111848004 | -101.54302055702918
> 6 | 0.13610415160655975 | 0.5015371441841125 | 0.46612346172332764 | 0.9201741218566895 | 0.8401027851458885 | 0.7178519111848004 | -101.54302055

### LAFTR for Eq Odds

In [13]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0025943517684936523 | 0.5033180713653564 | 0.4955350160598755 | 0.9201741218566895 | 0.6340351458885941 | 0.5579132514978593 | -101.54302055702918
> 2 | 0.011903762817382812 | 0.5005932450294495 | 0.464881956577301 | 0.9201741218566895 | 0.8289539124668435 | 0.7178519111848004 | -101.54302055702918
> 3 | 0.013974647037684917 | 0.5085198283195496 | 0.4665958881378174 | 0.9201741218566895 | 0.8370772546419099 | 0.7178519111848004 | -101.54302055702918
> 4 | 0.015777746215462685 | 0.5151233673095703 | 0.4677901268005371 | 0.9201741218566895 | 0.8395225464190982 | 0.7178519111848004 | -101.54302055702918
> 5 | 0.01719113253057003 | 0.519931972026825 | 0.46835857629776 | 0.9201741218566895 | 0.8412218169761273 | 0.7178519111848004 | -101.54302055702918
> 6 | 0.01826566457748413 | 0.5234362483024597 | 0.4686392545700073 | 0.9201741218566895 | 0.8429625331564987 | 0.7178519111848004 | -101.54302055

### LAFTR for Eq Opp

In [14]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        
        clas_acc, dp, deqodds, deqopp, confusion_matrix  = compute_metrics(Y, A, Y_hat, adim=adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]]

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | -0.006658988539129496 | 0.5038057565689087 | 0.5201970338821411 | 0.9201741218566895 | 0.6291860079575596 | 0.5497899091251649 | -101.54302055702918
> 2 | 0.010907616466283798 | 0.49299466609954834 | 0.4661451578140259 | 0.9201741218566895 | 0.8290368037135278 | 0.7178519111848004 | -101.54302055702918
> 3 | 0.01378756482154131 | 0.5000082850456238 | 0.46606963872909546 | 0.9201741218566895 | 0.8361240053050398 | 0.7178519111848004 | -101.54302055702918
> 4 | 0.015522480010986328 | 0.5044349431991577 | 0.4662257432937622 | 0.9201741218566895 | 0.8391080901856764 | 0.7178519111848004 | -101.54302055702918
> 5 | 0.01727452501654625 | 0.5089324712753296 | 0.46641063690185547 | 0.9201741218566895 | 0.8413461538461539 | 0.7178519111848004 | -101.54302055702918
> 6 | 0.01816876418888569 | 0.5112783908843994 | 0.46655529737472534 | 0.9201741218566895 | 0.8420507294429708 | 0.7178519111848004 | -101.5

## Saving into DF then CSV

In [15]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp
0,LAFTR4DP,0.2,0.850565,0.147926,0.691368,0.441297,0.252022,0.762748,0.581102
1,LAFTR4DP,0.5,0.850233,0.133836,0.687538,0.423316,0.231268,0.760279,0.565219
2,LAFTR4DP,0.7,0.850399,0.147661,0.695655,0.447747,0.251630,0.765283,0.586627
3,LAFTR4DP,1.0,0.849734,0.145136,0.694182,0.443133,0.247926,0.764122,0.582496
4,LAFTR4EqOdds,0.2,0.849568,0.145184,0.694149,0.443133,0.247989,0.764035,0.582457
5,LAFTR4EqOdds,0.5,0.849568,0.145184,0.694149,0.443133,0.247989,0.764035,0.582457
6,LAFTR4EqOdds,0.7,0.849568,0.145184,0.694149,0.443133,0.247989,0.764035,0.582457
7,LAFTR4EqOdds,1.0,0.849568,0.145184,0.694149,0.443133,0.247989,0.764035,0.582457
8,LAFTR4EqOpp,0.2,0.849235,0.150350,0.693593,0.446854,0.255471,0.763564,0.585583
9,LAFTR4EqOpp,0.5,0.849235,0.150350,0.693593,0.446854,0.255471,0.763564,0.585583


In [16]:
result_df.to_csv(f'{data_name}-result/laftr-{epochs}.csv')